In [41]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
from datetime import datetime
import os

In [ ]:
categories_links = ["republika-srpska", "vijesti/drustvo", "vijesti/region",  "vijesti/svijet", "vijesti/ekonomija", "ijesti/bih", "vijesti/banja-luka", "vijesti/srbija", "vijesti/gradovi-i-opstine"]


# Initialize an empty dataframe
df = pd.DataFrame(columns=['date', 'headline', 'author_name', 'cleaned_text', 'link', 'category', 'tags'])

for item in categories_links:
    for page_number in tqdm(range(1,500)):
        url = f'https://www.atvbl.rs/{item}?page={page_number}'
        response = requests.get(url)
        soup = bs(response.content, 'html.parser')
        div_container = soup.find('div', class_='row -mx-2 articles-list-container')
        links = [a['href'] for a in div_container.find_all('a', href=True)]
        for link in links:
            response_child = requests.get(link)
            soup_child = bs(response_child.content, 'html.parser')
            try:
                headline = soup_child.find('h1').text
            except:
                headline = np.nan
            break

        break
    break

headline

  0%|          | 0/499 [00:01<?, ?it/s]


'\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n'

In [ ]:
def serbian_str_to_datetime(data):
    months_serbian = {
        "јануара": "01",
        "фебруара": "02",
        "марта": "03",
        "априла": "04",
        "маја": "05",
        "јуна": "06",
        "јула": "07",
        "августа": "08",
        "септембра": "09",
        "октобра": "10",
        "новембра": "11",
        "децембра": "12"
    }

    day = data.split(',')[1].strip().split()[0]
    month_name = data.split(',')[1].strip().split()[1]
    year = data.split(',')[2].strip().split()[0]
    time = data.split('/')[1].strip()

    month = months_serbian.get(month_name, "")

    if not month:
        raise ValueError("Некорректное название месяца!")

    dt = datetime.strptime(f"{day} {month} {year} {time}", "%d %m %Y %H:%M")
    return dt



categories_links = ["gazeta/politika/", "gazeta/socijum/", "gazeta/svijet/",  "gazeta/ekonomija/", "gazeta/sport/", "magazin/kultura/", "magazin/tema-nedelje/", "magazin/nauka/", "magazin/scena/", "magazin/analize/"]

# Initialize an empty dataframe
df = pd.DataFrame(columns=['date', 'headline', 'author_name', 'cleaned_text', 'link', 'category', 'tags'])

"https://www.frontal.rs/k/gazeta/politika/page/2/"

for item in categories_links:
    for page_number in tqdm(range(1,500)):
        url = f'https://www.frontal.rs/k/{item}page/{page_number}/'
        response = requests.get(url)
        soup = bs(response.content, 'html.parser')
        div_container = soup.find('div', class_='col-md-8 mt-3')
        links = [a['href'] for a in div_container.find_all('a', href=True)][:-1]
        for link in links:
            response_child = requests.get(link)
            soup_child = bs(response_child.content, 'html.parser')
            try:
                headline = soup_child.find('h1').text
            except:
                headline = np.nan
            try:
                category = soup_child.find_all('div', class_='col-md-8')[0].text
                news_text = ""
                for p in soup_child.find('article').find_all('p'):
                    news_text = news_text + " " + p.text
            except:
                news_text = np.nan
            try:
                date = soup_child.find('figcaption', class_='figure-caption').text
                parced_date = serbian_str_to_datetime(date)
            except:
                parced_date

            try:
                tags_container = soup_child.find('span', class_='tags-links').find_all('a', href=True)
                tags = [tag.text for tag in tags_container]
            except:
                tags = np.nan
            
            temp_df = pd.DataFrame([{
                        'date': parced_date,
                        'headline': headline,
                        'author_name': np.nan,
                        'cleaned_text': news_text,
                        'link': link,
                        'category': category,
                        'tags': tags
                    }])

            df = pd.concat([df, temp_df], ignore_index=True)
            
        if parced_date < datetime(2023, 9, 28):
            break

# Save to temporary file after each category is processed
    temp_file_name = f"temp_data_{item}.csv"
    df.to_csv(temp_file_name, sep="|")

# Finally, save all data to the main file
file_path = os.path.join('Parser_rs_media/data', "nezavisne_data.csv")
df.to_csv(file_path, index=False, sep="|")


  0%|          | 1/499 [04:57<41:06:31, 297.17s/it]

In [171]:
df = pd.DataFrame(columns=['date', 'headline', 'author_name', 'cleaned_text', 'link', 'category', 'tags'])

date = datetime.now()
url ='https://www.rtrs.tv/index.php'
response = requests.get(url)
soup = bs(response.content, 'html.parser')
div_container = soup.find('div', class_='container main-width-fix z_fix')
link = div_container.find('a', href=True)
page_number = int(link['href'][link['href'].find("id=")+3:])
while date >= datetime(2023, 10, 4):
    url_p = f'https://www.rtrs.tv/vijesti/vijest.php?id={page_number}'
    response_p = requests.get(url_p)
    soup_p = bs(response_p.content, 'html.parser')
    div_container_p = soup_p.find('div', class_='col-md-8 col-sm-8')
    headline = div_container_p.find('h1').text if div_container_p.find('h1') else np.nan
    try:
        info_time_author =  div_container_p.find('div', class_='vrijeme-izvor').text if div_container_p.find('div', class_='vrijeme-izvor') else np.nan
        date = datetime.strptime(info_time_author[:10], '%d/%m/%Y')
    except:
        date
    
    try:
        author = info_time_author[info_time_author.find("Аутор")+6:].strip()
    except:
        author = np.nan

    try:    
        news_text = div_container_p.find('div', class_='lead').text
        nws_body = div_container_p.find('div', class_='nwzbody').find_all('p')
        for p in nws_body:
            news_text = news_text + p.text
    except:
        news_text = np.nan
        
    try:
        category = soup_p.find('div', class_='col-md-12 col-sm-12').find('li', class_='sel').text
    except:
        category = np.nan


    temp_df = pd.DataFrame([{
                    'date': date,
                    'headline': headline,
                    'author_name': author,
                    'cleaned_text': news_text,
                    'link': url_p,
                    'category': category,
                    'tags': np.nan
                }])
    
    df = pd.concat([df, temp_df], ignore_index=True)
    page_number -= 1

df.to_csv('rtrs_data.csv', index=False, sep="|") 


# file_path = os.path.join('Parser_rs_media/data', 'rtrs_data.csv')
# df.to_csv(file_path, index=False, sep="|") 

     

In [113]:

url = f'https://rtvbn.com/kategorija/vijesti/1'
response = requests.get(url)
soup = bs(response.content, 'html.parser')
div_container = soup.find('div', class_='tab-content')
links = [a['href'] for a in div_container.find_all('a', href=True)]
for i, link in enumerate(links):
    if i % 4 == 0:  
        response_child = requests.get(link)
        soup_child = bs(response_child.content, 'html.parser')
        try:
            headline = soup_child.find('h1').text
        except:
            headline = np.nan

        try:
            texts = soup_child.find_all('p', class_='txt')
            article = ""
            for item in texts:
                article = article + " " + item.text
        except:
            article = np.nan

        try:
            date = soup_child.find('span', class_='date').text.strip()
            parsed_date = datetime.strptime(date, '%d.%m.%Y | %H:%M')
        except:
            parsed_date = np.nan
        
        try:
            category = soup_child.find('strong').text.strip()
        except:
            category = np.nan

    break

    

        #     cleaned_text = '\n'.join(line.strip() for line in text.split('\n') if line.strip())


        

parsed_date

datetime.datetime(2023, 10, 4, 14, 1)

In [122]:
import requests
from bs4 import BeautifulSoup as bs
from tqdm import tqdm
import pandas as pd
from datetime import datetime
import numpy as np
import time

def scrape_rtvbn():

    df = pd.DataFrame(columns=['date', 'headline', 'author_name', 'cleaned_text', 'link', 'category', 'tags'])

    for page in tqdm(range(2000)):
        url = f'https://rtvbn.com/kategorija/vijesti/{page}'
        response = requests.get(url)
        soup = bs(response.content, 'html.parser')
        div_container = soup.find('div', class_='tab-content')
        links = [a['href'] for a in div_container.find_all('a', href=True)]
        
        for i, link in enumerate(links):
            if i % 4 == 0 and link:  
                response_child = requests.get(link)
                soup_child = bs(response_child.content, 'html.parser')

                headline = soup_child.find('h1').text if soup_child.find('h1') else np.nan
                texts = soup_child.find_all('p', class_='txt')
                article = " ".join(item.text for item in texts) if texts else np.nan
                date_text = soup_child.find('span', class_='date').text.strip() if soup_child.find('span', class_='date') else None
                category = soup_child.find('strong').text.strip() if soup_child.find('strong') else np.nan

                try:
                    parsed_date = datetime.strptime(date_text, '%d.%m.%Y | %H:%M')
                except:
                    parsed_date = np.nan

                temp_df = pd.DataFrame([{
                    'date': parsed_date,
                    'headline': headline,
                    'author_name': np.nan,
                    'cleaned_text': article,
                    'link': link,
                    'category': category,
                    'tags': np.nan
                }])
                df = pd.concat([df, temp_df], ignore_index=True)
                

        if parsed_date and parsed_date < datetime(2023, 10, 4):
            break

    df.to_csv("rtvbn_data.csv", sep="|")

if __name__ == "__main__":
    scrape_rtvbn()

  0%|          | 3/2000 [00:45<8:20:36, 15.04s/it]


In [123]:
df

,date,headline,author_name,cleaned_text,link,category,tags
0,04.10.2023 | 14:14,Narodni novac rasipa se i u Narodnoj skupštini,NaN,"Đorđe Vučinić, poslanik Liste za pravdu i red...",https://rtvbn.com/4050324/narodni-novac-rasipa...,Republika Srpska,NaN
1,04.10.2023 | 14:09,"Veliki požar u Osijeku, dim se vidi iz svemira...",NaN,On je objavio i fotografije iz mjesta Gradišt...,https://rtvbn.com/4050323/veliki-pozar-u-osije...,Region,NaN
2,04.10.2023 | 14:01,SNSD glasao protiv izmjena zakona o PDV-u,NaN,Na današnjoj sjednici Doma naroda Parlamentar...,https://rtvbn.com/4050322/snsd-glasao-protiv-i...,Republika Srpska,NaN
3,04.10.2023 | 13:52,Karan najavio: Slijedi čišćenje u redovima MUP RS,NaN,"“Čistićemo policiju, nećemo dozvoliti da ijed...",https://rtvbn.com/4050321/karan-najavio-slijed...,Republika Srpska,NaN
4,04.10.2023 | 13:18,"Dončić krenuo u štrajk, Trivićeva najavila pro...",NaN,- Poslanik Radislav Dončić (NF) počeo ispunja...,https://rtvbn.com/4050320/doncic-krenuo-u-stra...,Republika Srpska,NaN
5,04.10.2023 | 13:17,Iza Dodikovih izjava ne stoji srpski narod,NaN,- Srpski narod sigurno ne stoji iza takve izj...,https://rtvbn.com/4050319/iza-dodikovih-izjava...,Republika Srpska,NaN
6,04.10.2023 | 12:46,Crnadak traži zabranu duplih funkcija u Srpskoj,NaN,"“Poslanik može da bude doktor, može da operiš...",https://rtvbn.com/4050318/crnadak-trazi-zabran...,Republika Srpska,NaN
7,04.10.2023 | 11:42,Radnici upozorili Upravu šumarstva u Kneževu,NaN,Radnici Šumskog gazdinstva Čemernica iz Kneže...,https://rtvbn.com/4050316/radnici-upozorili-up...,Republika Srpska,NaN
8,04.10.2023 | 11:35,SDS očekuje da izmjene zakona o PDV-u budu usv...,NaN,On je izrazio nadu da delegati u Domu naroda ...,https://rtvbn.com/4050315/sds-ocekuje-da-izmje...,Republika Srpska,NaN
9,04.10.2023 | 11:26,Operativna akcija „Plug“ – slobode lišeno 13...,NaN,"Slobode je lišeno 13 lica, koja su sprovedena...",https://rtvbn.com/4050314/operativna-akcija-pl...,Republika Srpska,NaN
